<img align="left" style="width: 20%" src="https://drive.google.com/uc?export=view&id=0B2icrrA3UYBlOUtoc0NGS3hFQnM">


<br>
<br>
<br>

<div style="text-align: center; font-weight: bold; font-size: 250%;">
SQLAlchemy and Alembic
</div>

<br>
<br>
<br>

<div style="text-align: center"> SAP Labs Korea </div>
<div style="text-align: center"> 최영선 </div>

<br>
<br>


<img align="middle" src="https://2.bp.blogspot.com/-u5RzeZjX5Ew/T7CzT9vFgyI/AAAAAAAABJ8/fDbn24qYajg/s1600/Isaac-Newton.jpg" >

<div style="text-align: center">"If I have seen future it is by importing from the code of giants" </div>
<div style="text-align: right"> Definitely Not isaac Newton </div>

$ pip install yeongseon

In [1]:
import yeongseon

print(yeongseon.name)
print(yeongseon.job)

yeongseon choe
python infrastructure devloper


## Contens

### 1. Python DB API
### 2. SQLAlchemy
### 3. Alembic

## Python Database Programming 

#### Define Tables

| Users                        ||
| :------------ | :-----------: |
| user_id       | int           |
| user_name     | varchar(32)   |
| user_email    | varchar(32)   |

| Programs                     ||
| :------------ | :-----------: |
| program_id    | int           |
| program_name  | varchar(32)   |
| user_id       | varchar(32)   |

| Program_enrollment      ||
| :------------ | :-----------: |
| program_id    | int           |
| user_id       | int           |

## DBAPI

- PEP 0249 -- Python Database API Specification v2.0
- Python modules are used to access database
- There are many DBAPI implementations availabls

In [2]:
import sqlite3
conn = sqlite3.connect('dbapi.sqlite')

curs = conn.cursor()
sql_stmt = """
CREATE TABLE users (
user_id integer primary key autoincrement,
user_name text not null,
user_email text not null);
"""
curs.execute(sql_stmt)
conn.commit()

curs = conn.cursor()
sql_stmt = """
INSERT INTO users (user_name, user_email)
VALUES ('최영선', 'yeongseon.choe@pycon.kr')
"""
curs.execute(sql_stmt)
conn.commit()
conn.close()

$ pip install pymysql

## What is the SQLAlchemy ?


- The Database Toolkit for Python
- Object Relational Mapper (ORM)implementation.
- Introducted 2005
- end-to-end system for working with Python DBAPI, relational database, and the SQL language
- SQLAlchemy 1.2.0b2 released (Jul 2017)



## Why use SQLAlchemy
- SQLAlchemy provoices a generallized interface for creating and executing database-agnostic code without needing to write SQL statements.

## SQLAlchemyOverview

<img align="middle" src="http://docs.sqlalchemy.org/en/latest/_images/sqla_arch_small.png" >


### Installing SQLAlchemy

$ pip install sqlalchemy

### Version Check

In [3]:
import sqlalchemy

print( sqlalchemy.__version__)

1.1.11


### Connectiong

In [4]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///sqlalchemy.sqlite", echo=True, convert_unicode=True)

print(engine)

Engine(sqlite:///sqlalchemy.sqlite)


### Declaring a Mapping

- A Table that represents a table in a database
- A mapper that maps a Python class to a table in database
- A class object that defines how a database record maps to a normal Python object

#### Define Models

| Users                        ||
| :------------ | :-----------: |
| user_id       | int           |
| user_name     | varchar(32)   |
| user_email    | varchar(32)   |

In [5]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()

class User(Base):
    __tablename__ = "users"
    user_id = Column(Integer, primary_key=True,  autoincrement=True)
    user_name = Column(String(32), unique=True)
    user_email = Column(String(32), unique=True)
    
    def __init__(self, user_name, user_email):
        self.user_name = user_name
        self.user_email = user_email
    
    def __repr__(self):
        return "<User('%d', '%s, %s')>" % (self.user_id, self.user_name, self.user_email)

#### Basic Types

- Integer: basic integer type, generates INT
- String : ASCII strings, generated VARCHAR
- Unicode : Unicode strings, generates VARCHAR, NVARCHAR depending on database
- Boolean : generages BOOLEAN
- DateTime : generates DATATIME or TIMESTAMP
- Float : flating point values
- Numeric : precision numberics using Python

### Creating a Schema

In [6]:
User.__table__

Table('users', MetaData(bind=None), Column('user_id', Integer(), table=<users>, primary_key=True, nullable=False), Column('user_name', String(length=32), table=<users>), Column('user_email', String(length=32), table=<users>), schema=None)

In [7]:
User.__mapper__

<Mapper at 0x7fc9fdef03c8; User>

In [8]:
Base.metadata.create_all(bind=engine)

2017-08-09 21:54:01,466 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-08-09 21:54:01,467 INFO sqlalchemy.engine.base.Engine ()
2017-08-09 21:54:01,471 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-08-09 21:54:01,473 INFO sqlalchemy.engine.base.Engine ()
2017-08-09 21:54:01,480 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-08-09 21:54:01,483 INFO sqlalchemy.engine.base.Engine ()
2017-08-09 21:54:01,488 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	user_id INTEGER NOT NULL, 
	user_name VARCHAR(32), 
	user_email VARCHAR(32), 
	PRIMARY KEY (user_id), 
	UNIQUE (user_name), 
	UNIQUE (user_email)
)


2017-08-09 21:54:01,490 INFO sqlalchemy.engine.base.Engine ()
2017-08-09 21:54:01,536 INFO sqlalchemy.engine.base.Engine COMMIT


### Creating a Session

In [9]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()

print(session)

In [10]:
session.close()

### Adding and Updating Object

#### Adding Objects

In [11]:
Session = sessionmaker(bind=engine)

session = Session()

user = User("최영선", "yeongseon.choe@pycon.kr")

session.add(user)
session.commit()

2017-08-09 21:54:01,634 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:54:01,651 INFO sqlalchemy.engine.base.Engine INSERT INTO users (user_name, user_email) VALUES (?, ?)
2017-08-09 21:54:01,656 INFO sqlalchemy.engine.base.Engine ('최영선', 'yeongseon.choe@pycon.kr')
2017-08-09 21:54:01,672 INFO sqlalchemy.engine.base.Engine COMMIT


In [12]:
session.add_all([
    User("배준현", "junhyun.bae@pycon.kr"),
    User("김준기", "joongi.kim@pycon.kr")]
)
session.commit()

2017-08-09 21:54:01,729 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:54:01,740 INFO sqlalchemy.engine.base.Engine INSERT INTO users (user_name, user_email) VALUES (?, ?)
2017-08-09 21:54:01,744 INFO sqlalchemy.engine.base.Engine ('배준현', 'junhyun.bae@pycon.kr')
2017-08-09 21:54:01,757 INFO sqlalchemy.engine.base.Engine INSERT INTO users (user_name, user_email) VALUES (?, ?)
2017-08-09 21:54:01,761 INFO sqlalchemy.engine.base.Engine ('김준기', 'joongi.kim@pycon.kr')
2017-08-09 21:54:01,765 INFO sqlalchemy.engine.base.Engine COMMIT


In [13]:
user1 = User("강대성", "daesung.kang@pycon.kr")
user2 = User("한성준", "sungjun.han@pycon.kr")

session.bulk_save_objects([user1, user2])
session.commit()

2017-08-09 21:54:01,807 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:54:01,812 INFO sqlalchemy.engine.base.Engine INSERT INTO users (user_name, user_email) VALUES (?, ?)
2017-08-09 21:54:01,814 INFO sqlalchemy.engine.base.Engine (('강대성', 'daesung.kang@pycon.kr'), ('한성준', 'sungjun.han@pycon.kr'))
2017-08-09 21:54:01,829 INFO sqlalchemy.engine.base.Engine COMMIT


#### Updating Objects

In [14]:
user = session.query(User).filter_by().first() 
print(user)

2017-08-09 21:54:01,882 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:54:01,895 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users
 LIMIT ? OFFSET ?
2017-08-09 21:54:01,901 INFO sqlalchemy.engine.base.Engine (1, 0)
<User('1', '최영선, yeongseon.choe@pycon.kr')>


In [15]:
user.user_name = 'Yeongseon Choe'
print(user)
session.commit()

<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>
2017-08-09 21:54:01,924 INFO sqlalchemy.engine.base.Engine UPDATE users SET user_name=? WHERE users.user_id = ?
2017-08-09 21:54:01,931 INFO sqlalchemy.engine.base.Engine ('Yeongseon Choe', 1)
2017-08-09 21:54:01,940 INFO sqlalchemy.engine.base.Engine COMMIT


In [16]:
user = session.query(User).filter_by().first() 
print(user)

2017-08-09 21:54:01,990 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:54:01,994 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users
 LIMIT ? OFFSET ?
2017-08-09 21:54:01,999 INFO sqlalchemy.engine.base.Engine (1, 0)
<User('1', 'Yeongseon Choe, yeongseon.choe@pycon.kr')>


### Rolling Back

In [17]:
user = User("김태환", "taehwan.kim@pycon.kr")
session.add(user)

session.rollback()


2017-08-09 21:54:02,019 INFO sqlalchemy.engine.base.Engine ROLLBACK


### Querying

In [18]:
user = session.query(User).filter_by().first() 
user.user_name = '최영선'
session.commit()

2017-08-09 21:54:02,058 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:54:02,068 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users
 LIMIT ? OFFSET ?
2017-08-09 21:54:02,071 INFO sqlalchemy.engine.base.Engine (1, 0)
2017-08-09 21:54:02,079 INFO sqlalchemy.engine.base.Engine UPDATE users SET user_name=? WHERE users.user_id = ?
2017-08-09 21:54:02,081 INFO sqlalchemy.engine.base.Engine ('최영선', 1)
2017-08-09 21:54:02,096 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
for user in session.query(User):
    print(user)

2017-08-09 21:54:02,145 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:54:02,152 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users
2017-08-09 21:54:02,157 INFO sqlalchemy.engine.base.Engine ()
<User('1', '최영선, yeongseon.choe@pycon.kr')>
<User('2', '배준현, junhyun.bae@pycon.kr')>
<User('3', '김준기, joongi.kim@pycon.kr')>
<User('4', '강대성, daesung.kang@pycon.kr')>
<User('5', '한성준, sungjun.han@pycon.kr')>


In [20]:
for user in session.query(User).order_by(User.user_email):
    print(user)

2017-08-09 21:54:02,185 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users ORDER BY users.user_email
2017-08-09 21:54:02,191 INFO sqlalchemy.engine.base.Engine ()
<User('4', '강대성, daesung.kang@pycon.kr')>
<User('3', '김준기, joongi.kim@pycon.kr')>
<User('2', '배준현, junhyun.bae@pycon.kr')>
<User('5', '한성준, sungjun.han@pycon.kr')>
<User('1', '최영선, yeongseon.choe@pycon.kr')>


descending order

In [21]:
from sqlalchemy import desc
for user in session.query(User).order_by(desc(User.user_email)):
    print(user)

2017-08-09 21:54:02,225 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users ORDER BY users.user_email DESC
2017-08-09 21:54:02,233 INFO sqlalchemy.engine.base.Engine ()
<User('1', '최영선, yeongseon.choe@pycon.kr')>
<User('5', '한성준, sungjun.han@pycon.kr')>
<User('2', '배준현, junhyun.bae@pycon.kr')>
<User('3', '김준기, joongi.kim@pycon.kr')>
<User('4', '강대성, daesung.kang@pycon.kr')>


- fileter operators
    - equals <br>
    query(User).filetr(User.name == 'Yeongseon Choe')
    - not equals <br>
    query(User).filetr(User.name != 'Yeongseon Choe')
    - like <br>
    query(User).filetr(User.name.like('Yeongseon Choe')

In [22]:
print(session.query(User).filter(User.user_name == '최영선'))

SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?


In [23]:
session.query(User).filter(User.user_name == '최영선').all()

2017-08-09 21:54:02,289 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?
2017-08-09 21:54:02,297 INFO sqlalchemy.engine.base.Engine ('최영선',)


[<User('1', '최영선, yeongseon.choe@pycon.kr')>]

In [24]:
session.query(User).filter(User.user_name != '최영선').all()

2017-08-09 21:54:02,332 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name != ?
2017-08-09 21:54:02,337 INFO sqlalchemy.engine.base.Engine ('최영선',)


[<User('2', '배준현, junhyun.bae@pycon.kr')>,
 <User('3', '김준기, joongi.kim@pycon.kr')>,
 <User('4', '강대성, daesung.kang@pycon.kr')>,
 <User('5', '한성준, sungjun.han@pycon.kr')>]

In [25]:
session.query(User).filter(User.user_name.like('최%')).all()

2017-08-09 21:54:02,384 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name LIKE ?
2017-08-09 21:54:02,390 INFO sqlalchemy.engine.base.Engine ('최%',)


[<User('1', '최영선, yeongseon.choe@pycon.kr')>]

- fileter operators
    - in <br>
    query(User).filetr(User.name.in_(['최영선'])
    - not in <br>
    query(User).filetr(~User.name.in_(['최영선'])
    - is null <br>
    query(User).filetr(User.name == None)
    - is not null <br>
    query(User).filetr(User.name != None)

In [26]:
session.query(User).filter(User.user_name.in_(['최영선'])).all()

2017-08-09 21:54:02,421 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name IN (?)
2017-08-09 21:54:02,425 INFO sqlalchemy.engine.base.Engine ('최영선',)


[<User('1', '최영선, yeongseon.choe@pycon.kr')>]

In [27]:
session.query(User).filter(~User.user_name.in_(['최영선'])).all()

2017-08-09 21:54:02,461 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name NOT IN (?)
2017-08-09 21:54:02,464 INFO sqlalchemy.engine.base.Engine ('최영선',)


[<User('2', '배준현, junhyun.bae@pycon.kr')>,
 <User('3', '김준기, joongi.kim@pycon.kr')>,
 <User('4', '강대성, daesung.kang@pycon.kr')>,
 <User('5', '한성준, sungjun.han@pycon.kr')>]

In [28]:
session.query(User).filter(User.user_name == None).all()

2017-08-09 21:54:02,500 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name IS NULL
2017-08-09 21:54:02,508 INFO sqlalchemy.engine.base.Engine ()


[]

- fileter operators
    - textual SQL
    query(User).filetr(text("user_name == '최영선'"))

In [29]:
from sqlalchemy import text

session.query(User).filter(text("user_name == '최영선'")).all()

2017-08-09 21:54:02,569 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE user_name == '최영선'
2017-08-09 21:54:02,580 INFO sqlalchemy.engine.base.Engine ()


[<User('1', '최영선, yeongseon.choe@pycon.kr')>]

In [30]:
session.query(User).filter(User.user_name != None).all()

2017-08-09 21:54:02,655 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name IS NOT NULL
2017-08-09 21:54:02,678 INFO sqlalchemy.engine.base.Engine ()


[<User('1', '최영선, yeongseon.choe@pycon.kr')>,
 <User('2', '배준현, junhyun.bae@pycon.kr')>,
 <User('3', '김준기, joongi.kim@pycon.kr')>,
 <User('4', '강대성, daesung.kang@pycon.kr')>,
 <User('5', '한성준, sungjun.han@pycon.kr')>]

- Returning Lists and Scalars
    - all()
    - first()
    - one()

In [31]:
users = session.query(User).filter(User.user_name != '최영선').all()
for user in users:
   print(user) 

2017-08-09 21:54:02,754 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name != ?
2017-08-09 21:54:02,768 INFO sqlalchemy.engine.base.Engine ('최영선',)
<User('2', '배준현, junhyun.bae@pycon.kr')>
<User('3', '김준기, joongi.kim@pycon.kr')>
<User('4', '강대성, daesung.kang@pycon.kr')>
<User('5', '한성준, sungjun.han@pycon.kr')>


In [32]:
users = session.query(User).filter(User.user_name == '최영선').first()
print(users)

2017-08-09 21:54:02,819 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?
 LIMIT ? OFFSET ?
2017-08-09 21:54:02,832 INFO sqlalchemy.engine.base.Engine ('최영선', 1, 0)
<User('1', '최영선, yeongseon.choe@pycon.kr')>


In [33]:
users = session.query(User).filter(User.user_name == '최영선').one()
print(users)

2017-08-09 21:54:02,870 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?
2017-08-09 21:54:02,875 INFO sqlalchemy.engine.base.Engine ('최영선',)
<User('1', '최영선, yeongseon.choe@pycon.kr')>


### Counting

In [34]:
print(session.query(User).filter(User.user_name.like('%성%')).all())

session.query(User).filter(User.user_name.like('%성%')).count()

2017-08-09 21:54:02,910 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name LIKE ?
2017-08-09 21:54:02,920 INFO sqlalchemy.engine.base.Engine ('%성%',)
[<User('4', '강대성, daesung.kang@pycon.kr')>, <User('5', '한성준, sungjun.han@pycon.kr')>]
2017-08-09 21:54:02,931 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name LIKE ?) AS anon_1
2017-08-09 21:54:02,935 INFO sqlalchemy.engine.base.Engine ('%성%',)


2

### Deleting Object

In [35]:
user = session.query(User).filter(User.user_name == '최영선').first()
session.delete(user)
session.commit()

2017-08-09 21:54:02,953 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users 
WHERE users.user_name = ?
 LIMIT ? OFFSET ?
2017-08-09 21:54:02,957 INFO sqlalchemy.engine.base.Engine ('최영선', 1, 0)
2017-08-09 21:54:02,962 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.user_id = ?
2017-08-09 21:54:02,965 INFO sqlalchemy.engine.base.Engine (1,)
2017-08-09 21:54:02,978 INFO sqlalchemy.engine.base.Engine COMMIT


#### Define Models

| Users                        ||
| :------------ | :-----------: |
| user_id       | int           |
| user_name     | varchar(32)   |
| user_email    | varchar(32)   |

| Programs                     ||
| :------------ | :-----------: |
| program_id    | int           |
| program_name  | varchar(128)  |

| Program_enrollment      ||
| :------------ | :-----------: |
| program_entrollemnt_id |int   |
| program_id    | int           |
| user_id       | int           |

In [94]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()

class User(Base):
    __tablename__ = "users"
    user_id = Column(Integer, primary_key=True,  autoincrement=True)
    user_name = Column(String(32), unique=True)
    user_email = Column(String(32), unique=True)
    
    def __init__(self, user_name, user_email):
        self.user_name = user_name
        self.user_email = user_email
    
    def __repr__(self):
        return "<User('%d', '%s, %s')>" % (self.user_id, self.user_name, self.user_email)
    
class Program(Base):
    __tablename__ = "programs"
    program_id = Column(Integer, primary_key=True, autoincrement=True)
    program_name = Column(String(128), unique=True)
    
    def __init__(self, program_name):
        self.program_name = program_name
        
    def __repr__(self):
        return "<Program('%d', '%s')" % (self.program_id, self.program_name)
    
from sqlalchemy import ForeignKey
class ProgramEnrollment(Base):
    __tablename__ = "program_enrollments"
    program_enrollment_id = Column(Integer, primary_key=True, autoincrement=True)
    program_id = Column(Integer, ForeignKey("programs.program_id"))
    user_id = Column(Integer, ForeignKey("users.user_id"))
    
    def __init__(self, program_id, user_id):
        self.program_id = program_id
        self.user_id = user_id
    
    def __repr__(self):
        return "<ProgramEnrollment('%d', '%d', '%d')" % (self.program_enrollment_id, self.program_id, self.user_id)
    
Base.metadata.create_all(bind=engine)

2017-08-10 00:34:50,341 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2017-08-10 00:34:50,343 INFO sqlalchemy.engine.base.Engine ()
2017-08-10 00:34:50,346 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("programs")
2017-08-10 00:34:50,348 INFO sqlalchemy.engine.base.Engine ()
2017-08-10 00:34:50,351 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("program_enrollments")
2017-08-10 00:34:50,353 INFO sqlalchemy.engine.base.Engine ()


In [65]:
Session = sessionmaker(bind=engine)
session = Session()
session.add_all([        
    Program("SQLAlchemy and Alembic"),
    Program("얼렁뚱땅 파이썬 대소동"),
    Program("Meet aiotools: asyncio Idiom Library"),
    Program("PHP에서 Django로 갈아타기"),
    Program("파이썬을 통한 주식투자 보조시스템 만들기"),
    Program("니름: 쉬운 SOA 단위 테스트")]
)
session.commit()

2017-08-09 23:33:37,419 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 23:33:37,425 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name) VALUES (?)
2017-08-09 23:33:37,427 INFO sqlalchemy.engine.base.Engine ('SQLAlchemy and Alembic',)
2017-08-09 23:33:37,434 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name) VALUES (?)
2017-08-09 23:33:37,436 INFO sqlalchemy.engine.base.Engine ('얼렁뚱땅 파이썬 대소동',)
2017-08-09 23:33:37,438 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name) VALUES (?)
2017-08-09 23:33:37,441 INFO sqlalchemy.engine.base.Engine ('Meet aiotools: asyncio Idiom Library',)
2017-08-09 23:33:37,443 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name) VALUES (?)
2017-08-09 23:33:37,445 INFO sqlalchemy.engine.base.Engine ('PHP에서 Django로 갈아타기',)
2017-08-09 23:33:37,448 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name) VALUES (?)
2017-08-09 23:33:37,450 INFO sqlalchemy.engine

In [39]:
session.query(User).all()

2017-08-09 21:55:32,722 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:55:32,727 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users
2017-08-09 21:55:32,731 INFO sqlalchemy.engine.base.Engine ()


[<User('2', '배준현, junhyun.bae@pycon.kr')>,
 <User('3', '김준기, joongi.kim@pycon.kr')>,
 <User('4', '강대성, daesung.kang@pycon.kr')>,
 <User('5', '한성준, sungjun.han@pycon.kr')>]

In [40]:
session.query(Program).all()

2017-08-09 21:55:34,580 INFO sqlalchemy.engine.base.Engine SELECT programs.program_id AS programs_program_id, programs.program_name AS programs_program_name 
FROM programs
2017-08-09 21:55:34,589 INFO sqlalchemy.engine.base.Engine ()


[<Program('1', 'SQLAlchemy and Alembic'),
 <Program('2', '얼렁뚱땅 파이썬 대소동'),
 <Program('3', 'Meet aiotools: asyncio Idiom Library'),
 <Program('4', 'PHP에서 Django로 갈아타기'),
 <Program('5', '파이썬을 통한 주식투자 보조시스템 만들기'),
 <Program('6', '니름: 쉬운 SOA 단위 테스트')]

In [41]:
session.add_all([
    ProgramEnrollment(1, 2),
    ProgramEnrollment(1, 3),
    ProgramEnrollment(1, 4)]
)
session.commit()

2017-08-09 21:55:36,414 INFO sqlalchemy.engine.base.Engine INSERT INTO program_enrollments (program_id, user_id) VALUES (?, ?)
2017-08-09 21:55:36,418 INFO sqlalchemy.engine.base.Engine (1, 2)
2017-08-09 21:55:36,427 INFO sqlalchemy.engine.base.Engine INSERT INTO program_enrollments (program_id, user_id) VALUES (?, ?)
2017-08-09 21:55:36,430 INFO sqlalchemy.engine.base.Engine (1, 3)
2017-08-09 21:55:36,433 INFO sqlalchemy.engine.base.Engine INSERT INTO program_enrollments (program_id, user_id) VALUES (?, ?)
2017-08-09 21:55:36,436 INFO sqlalchemy.engine.base.Engine (1, 4)
2017-08-09 21:55:36,440 INFO sqlalchemy.engine.base.Engine COMMIT


In [42]:
session.query(ProgramEnrollment).all()

2017-08-09 21:55:38,812 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-09 21:55:38,817 INFO sqlalchemy.engine.base.Engine SELECT program_enrollments.program_enrollment_id AS program_enrollments_program_enrollment_id, program_enrollments.program_id AS program_enrollments_program_id, program_enrollments.user_id AS program_enrollments_user_id 
FROM program_enrollments
2017-08-09 21:55:38,822 INFO sqlalchemy.engine.base.Engine ()


[<ProgramEnrollment('1', '1', '2'),
 <ProgramEnrollment('2', '1', '3'),
 <ProgramEnrollment('3', '1', '4')]

### Querying with Joins

In [43]:
session.query(User).join(ProgramEnrollment).filter(ProgramEnrollment.program_id == 1).all()

2017-08-09 21:56:08,064 INFO sqlalchemy.engine.base.Engine SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users JOIN program_enrollments ON users.user_id = program_enrollments.user_id 
WHERE program_enrollments.program_id = ?
2017-08-09 21:56:08,068 INFO sqlalchemy.engine.base.Engine (1,)


[<User('2', '배준현, junhyun.bae@pycon.kr')>,
 <User('3', '김준기, joongi.kim@pycon.kr')>,
 <User('4', '강대성, daesung.kang@pycon.kr')>]

In [44]:
session.query(User).join(ProgramEnrollment).filter(ProgramEnrollment.program_id == 1).count()

2017-08-09 21:56:12,821 INFO sqlalchemy.engine.base.Engine SELECT count(*) AS count_1 
FROM (SELECT users.user_id AS users_user_id, users.user_name AS users_user_name, users.user_email AS users_user_email 
FROM users JOIN program_enrollments ON users.user_id = program_enrollments.user_id 
WHERE program_enrollments.program_id = ?) AS anon_1
2017-08-09 21:56:12,827 INFO sqlalchemy.engine.base.Engine (1,)


3

### Table schema changes ...

| Programs                     ||
| :------------ | :-----------: |
| program_id    | int           |
| program_name  | varchar(128)  |

| Programs                     ||
| :------------ | :-----------: |
| program_id    | int           |
| program_name  | varchar(128)  |
| user_id       | varchar(32)   |

## Alembic

### Database migration tool
- creation
- management
- invocation of change managment

### Installing Alembic

$ pip install alembic

### Usage
<pre>
usage: alembic [-h] [-c CONFIG] [-n NAME] [-x X] [--raiseerr]
               {branches,current,downgrade,edit,heads,history,init,
               list_templates,merge,revision,show,stamp,upgrade}     
</pre>

### Creating an Environment

<pre>
$ alembic init 'alembic' <br>
</pre>
```

project
│         alembic.ini
└──── alembic
      │    env.py
      │    README
      │    script.py.mako      
      └─ versions
```

  
$ vi alembic.ini

<pre>
# A generic, single database configuration.

[alembic]
# path to migration scripts
script_location = alembic

# template used to generate migration files
# file_template = %%(rev)s_%%(slug)s

# timezone to use when rendering the date
# within the migration file as well as the filename.
# string value is passed to dateutil.tz.gettz()
# leave blank for localtime
# timezone =

# max length of characters to apply to the
# "slug" field
#truncate_slug_length = 40

# set to 'true' to run the environment during
# the 'revision' command, regardless of autogenerate
# revision_environment = false

# set to 'true' to allow .pyc and .pyo files without
# a source .py file to be detected as revisions in the
# versions/ directory
# sourceless = false

# version location specification; this defaults
# to alembic/versions.  When using multiple version
# directories, initial revisions must be specified with --version-path
# version_locations = %(here)s/bar %(here)s/bat alembic/versions

# the output encoding used when revision files
# are written from script.py.mako
# output_encoding = utf-8
<span style="color:red">sqlalchemy.url = sqlite:///alembic.sqlite</span>

# Logging configuration
[loggers]
keys = root,sqlalchemy,alembic

[handlers]
keys = console

[formatters]
keys = generic

[logger_root]
level = WARN
handlers = console
qualname =

[logger_sqlalchemy]
level = WARN
handlers =
qualname = sqlalchemy.engine

[logger_alembic]
level = INFO
handlers =
qualname = alembic

[handler_console]
class = StreamHandler
args = (sys.stderr,)
level = NOTSET
formatter = generic

[formatter_generic]
format = %(levelname)-5.5s [%(name)s] %(message)s
datefmt = %H:%M:%S

</pre>

### Creating a Migration Script
<pre>
$ alembic revision -m "create tables"
</pre>
<br>
```
project
│         alembic.ini
└──── alembic
      │    env.py
      │    README
      │    script.py.mako      
      └─ versions
          │   506aac89f70c_create_tables.py
```

<pre>
$ vi alembic/versions/506aac89f70c_create_tables.py
</pre>

In [ ]:
"""create tables

Revision ID: 506aac89f70
Revises: 
Create Date: 2017-07-29 22:19:30.546010

"""
from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision = '506aac89f70'
down_revision = None
branch_labels = None
depends_on = None


def upgrade():
    pass


def downgrade():
    pass


#### Editing the migration script

In [80]:
"""create tables

Revision ID: 506aac89f70c
Revises:
Create Date: 2017-08-09 22:14:41.836325

"""
from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision = '506aac89f70c'
down_revision = None
branch_labels = None
depends_on = None


def upgrade():
    op.create_table(
        "users",
        sa.Column("user_id", sa.Integer, primary_key=True, autoincrement=True),    
        sa.Column("user_name", sa.String(32), unique=True),
        sa.Column("user_email", sa.String(32), unique=True),
    )

    op.create_table(
        "programs",
        sa.Column("program_id", sa.Integer, primary_key=True, autoincrement=True),
        sa.Column("program_name", sa.String(128), unique=True),
    )

    op.create_table(
        "program_enrollments",
        sa.Column("program_enrollment_id", sa.Integer, primary_key=True, autoincrement=True),
        sa.Column("program_id", sa.Integer, sa.ForeignKey("programs.program_id")),
        sa.Column("user_id", sa.Integer, sa.ForeignKey("users.user_id")),
    )

def downgrade():
    op.drop_table("users")
    op.drop_table("programs")
    op.drop_table("program_enrollments")
    

 <pre>
 $ alembic.exe upgrade head
 
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 506aac89f70c, create tables
</pre>

In [95]:
engine = create_engine("sqlite:///alembic.sqlite", echo=True, convert_unicode=True)


In [96]:
Session = sessionmaker(bind=engine)
session = Session()
session.add_all([        
    Program("SQLAlchemy and Alembic"),
    Program("얼렁뚱땅 파이썬 대소동"),
    Program("Meet aiotools: asyncio Idiom Library")]
)
session.commit()

2017-08-10 00:35:02,717 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2017-08-10 00:35:02,721 INFO sqlalchemy.engine.base.Engine ()
2017-08-10 00:35:02,724 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2017-08-10 00:35:02,727 INFO sqlalchemy.engine.base.Engine ()
2017-08-10 00:35:02,731 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-10 00:35:02,737 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name) VALUES (?)
2017-08-10 00:35:02,740 INFO sqlalchemy.engine.base.Engine ('SQLAlchemy and Alembic',)
2017-08-10 00:35:02,748 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name) VALUES (?)
2017-08-10 00:35:02,751 INFO sqlalchemy.engine.base.Engine ('얼렁뚱땅 파이썬 대소동',)
2017-08-10 00:35:02,754 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name) VALUES (?)
2017-08-10 00:35:02,756 INFO sqlalchemy.engine.base.Engine ('Meet aiotools: asyn

<pre>
$ alembic revision -m "Add a column"
</pre>
<br>
```
project
│         alembic.ini
└──── alembic
      │    env.py
      │    README
      │    script.py.mako      
      └─ versions
          │   506aac89f70c_create_tables.py
          │   aadc5fd970b1_add_a_column.py
```

In [ ]:
"""Add a column

Revision ID: aadc5fd970b1
Revises: 506aac89f70c
Create Date: 2017-08-09 23:35:11.905893

"""
from alembic import op
import sqlalchemy as sa


# revision identifiers, used by Alembic.
revision = 'aadc5fd970b1'
down_revision = '506aac89f70c'
branch_labels = None
depends_on = None


def upgrade():
    op.add_column(
        "programs",        
        sa.Column("user_id", sa.Integer, primary_key=True, nullable=True),
    )


def downgrade():
    pass


<pre>
$ alembic current
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
506aac89f70c

$ alembic upgrade head
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade 506aac89f70c -> aadc5fd970b1, Add a column

$ alembic history
506aac89f70c -> aadc5fd970b1 (head), Add a column
<base> -> 506aac89f70c, create tables

</pre>

In [97]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()

class Program(Base):
    __tablename__ = "programs"
    program_id = Column(Integer, primary_key=True, autoincrement=True)
    program_name = Column(String(128), unique=True)
    user_id = Column(Integer, primary_key=True)
    
    def __init__(self, program_name, user_id):
        self.program_name = program_name
        self.user_id = user_id
        
    def __repr__(self):
        return "<Program('%d', '%s', '%d')" % (self.program_id, self.program_name, self.user_id)

In [98]:
Session = sessionmaker(bind=engine)
session = Session()
session.add_all([        
    Program("PHP에서 Django로 갈아타기", 4),
    Program("파이썬을 통한 주식투자 보조시스템 만들기", 5),
    Program("니름: 쉬운 SOA 단위 테스트", 6)]
)
session.commit()


2017-08-10 00:35:14,595 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2017-08-10 00:35:14,600 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name, user_id) VALUES (?, ?)
2017-08-10 00:35:14,603 INFO sqlalchemy.engine.base.Engine ('PHP에서 Django로 갈아타기', 4)
2017-08-10 00:35:14,612 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name, user_id) VALUES (?, ?)
2017-08-10 00:35:14,614 INFO sqlalchemy.engine.base.Engine ('파이썬을 통한 주식투자 보조시스템 만들기', 5)
2017-08-10 00:35:14,616 INFO sqlalchemy.engine.base.Engine INSERT INTO programs (program_name, user_id) VALUES (?, ?)
2017-08-10 00:35:14,618 INFO sqlalchemy.engine.base.Engine ('니름: 쉬운 SOA 단위 테스트', 6)
2017-08-10 00:35:14,621 INFO sqlalchemy.engine.base.Engine COMMIT


In [101]:
session.query(Program).filter(Program.user_id < 3).all()

2017-08-10 00:37:34,174 INFO sqlalchemy.engine.base.Engine SELECT programs.program_id AS programs_program_id, programs.program_name AS programs_program_name, programs.user_id AS programs_user_id 
FROM programs 
WHERE programs.user_id < ?
2017-08-10 00:37:34,177 INFO sqlalchemy.engine.base.Engine (3,)


[]

In [100]:
session.query(Program).filter(Program.user_id > 3).all()

2017-08-10 00:37:14,937 INFO sqlalchemy.engine.base.Engine SELECT programs.program_id AS programs_program_id, programs.program_name AS programs_program_name, programs.user_id AS programs_user_id 
FROM programs 
WHERE programs.user_id > ?
2017-08-10 00:37:14,943 INFO sqlalchemy.engine.base.Engine (3,)


[<Program('4', 'PHP에서 Django로 갈아타기', '4'),
 <Program('5', '파이썬을 통한 주식투자 보조시스템 만들기', '5'),
 <Program('6', '니름: 쉬운 SOA 단위 테스트', '6')]

In [63]:
import os

os.remove('dbapi.sqlite')
os.remove('sqlalchemy.sqlite')
#os.remove('alembic.sqlite')

<br>
<br>
<br>
<div style="text-align: center; font-weight: bold; font-size: 250%;">
Q&A 
</div>
<br>
<br>
<br>


<br>
<br>
<br>
<div style="text-align: center; font-weight: bold; font-size: 250%;">
Happy Pythoning!
</div>
<br>
<br>
<br>